In [1]:
import numpy as np
import nltk
import string
from nltk import bigrams
from collections import Counter, defaultdict

In [2]:
toy_corpus = ["This is the house that Jack built", "This is the malt", 
              "That lay in the house that Jack built", "This is the rat",
             "That ate the malt", "That lay in the house that Jack built",
             "This is the cat", "That killed the rat", "That ate the malt", 
             "That lay in the house that Jack built"]

# Demonstration of bigrams working

In [3]:
for sents in toy_corpus:
    sents = sents.lower()
    sents = f"<start> {sents} <end>"
    for w1, w2 in bigrams(sents.split()):
        print(w1, w2)
    print("-"*10)

<start> this
this is
is the
the house
house that
that jack
jack built
built <end>
----------
<start> this
this is
is the
the malt
malt <end>
----------
<start> that
that lay
lay in
in the
the house
house that
that jack
jack built
built <end>
----------
<start> this
this is
is the
the rat
rat <end>
----------
<start> that
that ate
ate the
the malt
malt <end>
----------
<start> that
that lay
lay in
in the
the house
house that
that jack
jack built
built <end>
----------
<start> this
this is
is the
the cat
cat <end>
----------
<start> that
that killed
killed the
the rat
rat <end>
----------
<start> that
that ate
ate the
the malt
malt <end>
----------
<start> that
that lay
lay in
in the
the house
house that
that jack
jack built
built <end>
----------


# Define the model

In [4]:
model = defaultdict(lambda: defaultdict(lambda: 0)) # This will be dictionary of 
                                                    # dictionary where each value of child dictionary
                                                    # will be initialized to 0

In [5]:
for sents in toy_corpus:
    sents = sents.lower()
    sents = f"<start> {sents} <end>" # Here <start> and <end> are the fake tokens that will help to identify
                                     # the sentence limits
    for w1, w2 in bigrams(sents.split()):
        model[w1][w2] += 1 # Get the raw count of each bigram tokens

## Explanation of the above code:

The bigram tokens of our toy_corpus is shown above, what the above piece of code does is simple it calculates the count of bigram tokens as follows :

```
{
   That : { Jack : 4, lay : 3, ate: 2, killed : 1}, 
   This : {is : 4}
}
```

In [6]:
# Let's see the model if it verifies the above explanation
model

defaultdict(<function __main__.<lambda>()>,
            {'<start>': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'this': 4, 'that': 6}),
             'this': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'is': 4}),
             'is': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'the': 4}),
             'the': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'house': 4, 'malt': 3, 'rat': 2, 'cat': 1}),
             'house': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'that': 4}),
             'that': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'jack': 4, 'lay': 3, 'ate': 2, 'killed': 1}),
             'jack': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'built': 4}),
             'built': defau

Looks like what I explained it really reflects what's done by the code

Moving on to the <b>Bigram Model</b>


# Bigram Model

$$
\begin{equation}
p(w) = \prod_{i=1}^{k+1} p(w_{i} | w_{i-1})
\end{equation}
$$

where,

$$
\begin{equation}
p(w_i | w_{i-1}) = \frac{c(w_{i-1}w_{i})}{\sum_{w_{i}} c(w_{i-1}w_{i})} 
                 = \frac{c(w_{i-1}w_{i})}{c(w_{i-1})}
\end{equation}
$$


To explain the equation (2) let's consider the following example:

```
{
   that : { jack : 4, lay : 3, ate: 2, killed : 1}, 
   this : {is : 4}
}
```

If we are to calculate the probability of `p(ate | that)` we will do the following calculation

$$
\begin{equation}
p(ate | that) = \frac{c(that \ ate)}{c(that)} = \frac{2}{10}
\end{equation}
$$

Now the above scenario looks pretty decent but what does $\frac{c(w_{i-1}w_{i})}{\sum_{w_{i}} c(w_{i-1}w_{i})}$   mean???? 

Well the summation term at the end means that the word $w_{i-1}$, in our case it's `that`, occurs with different words and their count i.e. 
```
that : { jack : 4, lay : 3, ate: 2, killed : 1}

```

Now if we sum all those values we will get $10$ as a result which is equal to the denominator in equation (3).

In [7]:
# As per the previous explanation let's built the model.

for w1 in model:
    total_count = float(sum(model[w1].values())) # equation (2) summation denominator operation
    for w2 in model[w1]:
        model[w1][w2] /= total_count # Converting them into probabilities
        
# Let's see the model now 
model

defaultdict(<function __main__.<lambda>()>,
            {'<start>': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'this': 0.4, 'that': 0.6}),
             'this': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'is': 1.0}),
             'is': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'the': 1.0}),
             'the': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'house': 0.4, 'malt': 0.3, 'rat': 0.2, 'cat': 0.1}),
             'house': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'that': 1.0}),
             'that': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'jack': 0.4, 'lay': 0.3, 'ate': 0.2, 'killed': 0.1}),
             'jack': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'built': 1.0}),

# Let's see what kind of text our model generates

In [8]:
start_tokens = "this"
sents = []
sents.append(start_tokens)

finished = False

while not finished:
    tokens = [(k,v) for k,v in sorted(model[start_tokens].items(), key=lambda x: x[1], reverse=True)]
    if len(tokens) == 0:
        break
        
    next_word = tokens[0][0]
    sents.append(next_word)
    start_tokens = next_word
    
    if start_tokens == "<end>":
        finished = True
        
print(sents)

['this', 'is', 'the', 'house', 'that', 'jack', 'built', '<end>']


In [9]:
finished_sents = ""

for tokens in sents:
    
    if tokens in ["<start>", "<end>"]:
        continue
    
    if tokens not in string.punctuation:
        finished_sents += " "+ tokens
        
    else:
        finished_sents += tokens
        
        
print(f"Final Sentence is : {finished_sents}")

Final Sentence is :  this is the house that jack built
